In [5]:
import idaes_examples

ModuleNotFoundError: No module named 'idaes_examples'

In [1]:
###############################################################################
# The Institute for the Design of Advanced Energy Systems Integrated Platform
# Framework (IDAES IP) was produced under the DOE Institute for the
# Design of Advanced Energy Systems (IDAES).
#
# Copyright (c) 2018-2024 by the software owners: The Regents of the
# University of California, through Lawrence Berkeley National Laboratory,
# National Technology & Engineering Solutions of Sandia, LLC, Carnegie Mellon
# University, West Virginia University Research Corporation, et al.
# All rights reserved.  Please see the files COPYRIGHT.md and LICENSE.md
# for full copyright and license information.
###############################################################################

from enum import Enum
import pandas as pd
import numpy as np

import pyomo.environ as pyo
from pyomo.common.collections import ComponentSet, ComponentMap
from pyomo.util.calc_var_value import calculate_variable_from_constraint

import idaes.core.util.scaling as iscale
from pyomo.dae import DerivativeVar
from idaes.core.solvers import petsc
import idaes.logger as idaeslog
import idaes.core.util.model_serializer as ms
from idaes_examples.mod.power_gen.soc_dynamic_flowsheet import SocStandaloneFlowsheet as SocFlowsheet
import matplotlib.pyplot as plt
from idaes.models.control.controller import ControllerType, ControllerMVBoundType, ControllerAntiwindupType
from IPython.display import SVG, display

ModuleNotFoundError: No module named 'idaes_examples'

In [ ]:
class OperatingScenario(Enum):
    maximum_production = 1
    power_mode = 2

def scale_indexed_constraint(con, sf):
    for idx, c in con.items():
        iscale.constraint_scaling_transform(c, sf)

def set_indexed_variable_bounds(var, bounds):
    for idx, subvar in var.items():
        subvar.bounds = bounds

def create_ramping_eqns(fs, vars, time_scaling_factor=1):
    def rule_ramp(b, t, dvdt, v_ramp):
        return dvdt[t] == v_ramp[t]
    t0 = fs.time.first()

    for var in vars:
        var.unfix()
        shortname = var.name.split(".")[-1]
        blk = var.parent_block()
        dvdt = DerivativeVar(var, wrt=fs.time, initialize=0)
        setattr(blk, "d" + shortname + "dt", dvdt)
        v_ramp = pyo.Var(fs.time, initialize=0)
        setattr(blk, shortname + "_ramp_rate", v_ramp)
        v_ramp_eqn = pyo.Constraint(fs.time, rule=lambda b, t: rule_ramp(b, t, dvdt, v_ramp))
        setattr(blk, shortname + "_ramp_rate_eqn", v_ramp_eqn)
        for t in fs.time:
            sv = iscale.get_scaling_factor(var[t], default=1)
            iscale.set_scaling_factor(dvdt[t], sv/time_scaling_factor)
            iscale.set_scaling_factor(v_ramp[t], sv/time_scaling_factor)
            iscale.constraint_scaling_transform(v_ramp_eqn[t], sv/time_scaling_factor)

        v_ramp_eqn[t0].deactivate()
        v_ramp[t0].fix(0)

In [ ]:
operating_scenario = OperatingScenario.maximum_production
m = pyo.ConcreteModel()
t_start = 1 * 60 * 60
t_ramp = 5 * 60
t_settle = 2 * 60 * 60
t_end = 3 * 60 * 60

dt_set = [t_start, t_ramp, t_settle, t_ramp, t_end]

time_set = [sum(dt_set[:j]) for j in range(len(dt_set)+1)]

# The names here correspond to the row names in 
# soec_flowsheet_operating_conditions.csv
# There should be len(time_set) entries here.
# We start simulating a period at maximum production
# in order to confirm the system is at steady state.
if operating_scenario == OperatingScenario.maximum_production:
    setpoints = [
        "maximum_H2", "maximum_H2",
        "power", "power",
        "maximum_H2", "maximum_H2",
    ]
elif operating_scenario == OperatingScenario.power_mode: 
    setpoints = [
        "power", "power",
        "maximum_H2", "maximum_H2",
        "power", "power",
    ]
else:
    raise RuntimeError("Please choose a valid operating scenario")

m.fs = SocFlowsheet(
    dynamic=True,
    time_set=time_set,
    time_units=pyo.units.s,
    thin_electrolyte_and_oxygen_electrode=True,
    include_interconnect=True,
)
scaling_log = idaeslog.getLogger("idaes.core.util.scaling")
scaling_log.setLevel(idaeslog.ERROR)
iscale.calculate_scaling_factors(m)

In [ ]:
for t in m.fs.time:
    m.fs.condenser_flash.control_volume.properties_in[t].flow_mol_phase["Liq"].domain = pyo.Reals
    m.fs.condenser_flash.control_volume.properties_in[t].flow_mol_phase["Liq"].bounds = (None, None)
    m.fs.condenser_flash.control_volume.properties_in[t].phase_frac["Liq"].domain = pyo.Reals
    m.fs.condenser_flash.control_volume.properties_in[t].phase_frac["Liq"].bounds = (None, None)
    for var in [
        m.fs.condenser_flash.control_volume.properties_in[t].log_mole_frac_tdew, 
        m.fs.condenser_flash.control_volume.properties_in[t]._mole_frac_tdew,
    ]:
        for idx in var.index_set():
            var[idx].domain = pyo.Reals
            var[idx].bounds = (None, None)